### Data Format Conversion: json > csv

In [15]:
import pandas as pd
import os
import json
import numpy as np
import re

In [19]:
# Filter irrelevant characters and links
def sentence_process(string):
    
    word_list = string.split(' ')
    new_list = []
    for word in word_list:
        if word.startswith('@') or ('http' in word) or ('@' in word):
            continue
        else:
            new_list.append(word)
    final_string = ' '.join(new_list)
    if not final_string.endswith('\n'):
        final_string += '\n'

    # Remove emojis using regex
    final_string = re.sub(r'[^\w\s,.\-?!\'"]', '', final_string)
    
    return final_string


In [26]:
title = ['text_comments','text_only','comments_only','label','count']
csv_list = []
# event_list = ['charliehebdo','ferguson','germanwings-crash','ottawashooting','sydneysiege']
event_list = ['bostonbombings']
# base_path ='./data/pheme-rnr'
# base_path ='./TwitterDataset/phemernrdataset/pheme-rnr-dataset/'
base_path ='/home/angelos/MyThesis/PHEME-RNR/TwitterDataset/phemernrdataset/pheme-rnr-dataset/'

In [42]:
for event in event_list:
    news_path = os.path.join(base_path, event)
    rumour_path = os.path.join(news_path,'rumours')
    nonrumour_path = os.path.join(news_path,'non-rumours')

    for rumour_tweet in os.listdir((rumour_path)):
        full_path = os.path.join(rumour_path, rumour_tweet)
        if rumour_tweet.startswith("._") or not os.path.isdir(full_path):
            continue
            
        text_comments = ""
        text_only = ""
        comments_only = ""
        count = 0
        
        rumour_text = os.path.join(rumour_path, rumour_tweet, 'source-tweets')
        for tweet_text in os.listdir(rumour_text):
            if tweet_text.startswith("._") or not tweet_text.endswith(".json"):
                continue  # skip metadata or invalid files
            json_path = os.path.join(rumour_text,tweet_text)
            with open(json_path, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                text_comments += sentence_process(json_data['text'])
                text_comments += '[SEP]'
                text_only += sentence_process(json_data['text'])
                text_only += '[SEP]'

        rumour_comment = os.path.join(rumour_path, rumour_tweet, 'reactions')
        for comment in os.listdir(rumour_comment):
            if comment.startswith("._") or not comment.endswith(".json"):
                continue  # skip metadata or invalid files
            json_path = os.path.join(rumour_comment, comment)
            with open(json_path, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                text = json_data.get('full_text') or json_data.get('text')
                if text and sentence_process(text) != '\n':
                    text_comments += sentence_process(text)
                    text_comments += '[SEP]'
                    comments_only += sentence_process(text)
                    comments_only += '[SEP]'
                    count += 1

        single_list = [text_comments, text_only, comments_only, 'rumour', count]
        csv_list.append(single_list)

            
    for nonrumour_tweet in os.listdir((nonrumour_path)):
        full_path = os.path.join(nonrumour_path, nonrumour_tweet)
        if nonrumour_tweet.startswith("._") or not os.path.isdir(full_path):
            continue
            
        text_comments = ""
        text_only = ""
        comments_only = ""
        count = 0

        nonrumour_text = os.path.join(nonrumour_path, nonrumour_tweet, 'source-tweets')
        for tweet_text in os.listdir(nonrumour_text):
            if tweet_text.startswith("._") or not tweet_text.endswith(".json"):
                continue  # skip metadata or invalid files
            json_path = os.path.join(nonrumour_text,tweet_text)
            with open(json_path, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                text_comments += sentence_process(json_data['text'])
                text_comments += '[SEP]'
                text_only += sentence_process(json_data['text'])
                text_only += '[SEP]'

        nonrumour_comment = os.path.join(nonrumour_path, nonrumour_tweet, 'reactions')
        for comment in os.listdir(nonrumour_comment):
            if comment.startswith("._") or not comment.endswith(".json"):
                continue  # skip metadata or invalid files
            json_path = os.path.join(nonrumour_comment, comment)
            with open(json_path, 'r', encoding='utf-8') as file:
                json_data = json.load(file)
                text = json_data.get('full_text') or json_data.get('text')
                if text and sentence_process(text) != '\n':
                    text_comments += sentence_process(text)
                    text_comments += '[SEP]'
                    comments_only += sentence_process(text)
                    comments_only += '[SEP]'
                    count += 1

        single_list = [text_comments,text_only, comments_only, 'nonrumour',count]
        csv_list.append(single_list)


In [43]:
csv_file=pd.DataFrame(columns=title,data=csv_list)
csv_file.to_csv('./data/data_for_testing.csv',index = 0)